In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime

In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

In [3]:
start_year = 2015
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['PREC_ACC_NC'].dims[1:], coords={'south_north': ds['PREC_ACC_NC'].coords['south_north'], 'west_east': ds['PREC_ACC_NC'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s.PREC_ACC_NC.values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'p': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)
                        

2024-10-06 17:11:47
2024-10-06 17:24:42
2024-10-06 17:36:50
2024-10-06 17:49:51
2024-10-06 18:02:57
2024-10-06 18:14:56
2024-10-06 18:27:43
2024-10-06 18:39:55
2024-10-06 18:53:24
2024-10-06 19:05:45
2024-10-06 19:19:26
2024-10-06 19:32:29
2024-10-06 19:45:02
2024-10-06 19:58:01
2024-10-06 20:12:24
2024-10-06 20:25:35
2024-10-06 20:39:03
2024-10-06 20:51:28
2024-10-06 21:05:25
2024-10-06 21:18:53
2024-10-06 21:32:07
2024-10-06 21:44:19
2024-10-06 21:57:03
2024-10-06 22:10:07


In [2]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 2015
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:42:24
2024-11-02 20:56:17
2024-11-02 21:10:04
2024-11-02 21:24:24
2024-11-02 21:39:04
2024-11-02 21:52:53
2024-11-02 22:07:19
2024-11-02 22:20:30
2024-11-02 22:35:18
2024-11-02 22:48:55
2024-11-02 23:03:47
2024-11-02 23:18:10
2024-11-02 23:31:58
2024-11-02 23:45:55
2024-11-02 23:59:05
2024-11-03 00:12:43
2024-11-03 00:26:27
2024-11-03 00:38:52
2024-11-03 00:52:25
2024-11-03 01:06:17
2024-11-03 01:20:29
2024-11-03 01:35:09
2024-11-03 01:49:26
2024-11-03 01:03:57


In [3]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 2015
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 01:17:17
2024-11-03 01:32:56
2024-11-03 01:46:46
2024-11-03 02:02:05
2024-11-03 02:16:54
2024-11-03 02:30:34
2024-11-03 02:43:49
2024-11-03 02:57:29
2024-11-03 03:11:18
2024-11-03 03:24:55
2024-11-03 03:38:55
2024-11-03 03:53:22
2024-11-03 04:06:39
2024-11-03 04:21:14
2024-11-03 04:34:59
2024-11-03 04:49:14
2024-11-03 05:03:02
2024-11-03 05:15:39
2024-11-03 05:29:04
2024-11-03 05:41:32
2024-11-03 05:54:20
2024-11-03 06:06:34
2024-11-03 06:19:28
2024-11-03 06:28:01


In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 2021
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:45:01
2025-02-17 01:45:34
2025-02-17 01:46:04
2025-02-17 01:46:31
2025-02-17 01:47:03
2025-02-17 01:47:29
2025-02-17 01:47:57
2025-02-17 01:48:26
2025-02-17 01:48:51
2025-02-17 01:49:28
2025-02-17 01:49:59
2025-02-17 01:50:27
2025-02-17 01:50:53
2025-02-17 01:51:22
2025-02-17 01:51:48
2025-02-17 01:52:16
2025-02-17 01:52:41
2025-02-17 01:53:06
2025-02-17 01:53:34
2025-02-17 01:54:00
2025-02-17 01:54:28
2025-02-17 01:54:55
2025-02-17 01:55:25
2025-02-17 01:55:56
2025-02-17 01:56:24
2025-02-17 01:56:48
2025-02-17 01:57:13
2025-02-17 01:57:36
2025-02-17 01:57:54
2025-02-17 01:58:17
2025-02-17 01:58:43
2025-02-17 01:59:11
2025-02-17 01:59:15
2025-02-17 01:59:41
2025-02-17 02:00:10
2025-02-17 02:00:36
2025-02-17 02:01:03
2025-02-17 02:01:32
2025-02-17 02:02:01
2025-02-17 02:02:29
2025-02-17 02:02:54
2025-02-17 02:03:27
2025-02-17 02:03:56
2025-02-17 02:04:19
2025-02-17 02:04:40
2025-02-17 02:05:02
2025-02-17 02:05:28
2025-02-17 02:05:52
2025-02-17 02:06:19
2025-02-17 02:06:44


OSError: no files to open